# Consultas em SQL

### Tema: Gerenciador de peladas
#### Grupo:
* Breno Poggiali de Sousa
* Gabriela Peres Neme
* Guilherme de Abreu Lima Buitrago Miranda
* Vinicius Julião Ramos 

#### Objetivo:
Esse notebook abarca parte do trabalho prático que solicita a criação de 10 consutlas SQL, contendo descrição textual, comando SQL e resultado.

Tais consultas devem atender aos requisitos mínimos:
* 2 operações de seleção e projeção
* 3 junção de duas relações
* 3 junção de três ou mais relações
* 2 funções de agregação sobre o resultado da junção de pelo menos duas relações

Ademais, o presente Notebook incluirá o item iii das Características avançadas, que especifica:
* iii. mais três consultas do tipo relatório

#### Importação das bibliotecas:

In [2]:
import io
import sqlite3
import pandas as pd

#### Conexão com o banco de dados:

In [3]:
conn = sqlite3.connect('BD/tp2.db')
cursor = conn.cursor()

#### Carga dos dados:
Código comentado pois deve ser executado uma única vez, e já o foi.

In [3]:
#f = io.open('BD/criacao_tabelas.sql', 'r', encoding = 'utf-8')
#sql = f.read()
#cursor.executescript(sql)

#f = io.open('BD/popula_tabelas.sql', 'r', encoding = 'utf-8')
#sql = f.read()
#cursor.executescript(sql)

## 10 consultas SQL

#### Consulta 1
Lista, para uma vaquinha coletiva específica, todas as pessoas que estão participando e o valor já pago por cada um dos participantes.

In [4]:
id_vaquinha_coletiva = 2

df01 = pd.read_sql_query("SELECT nome_pessoa, valor_pago " +
                         "FROM Vaquinha_Coletiva_Pessoa NATURAL JOIN Pessoa " +
                         "WHERE id_vaquinha_coletiva = " + str(id_vaquinha_coletiva) + " "
                         "ORDER BY nome_pessoa;", conn)

df01

,nome_pessoa,valor_pago
0,Beatriz,20.0
1,Cecília,20.0
2,Davi,20.0
3,Emanuelly,20.0
4,Giovanna,20.0
5,Helena,20.0
6,Heloisa,20.0
7,Isadora,20.0
8,Lara,0.0
9,Lorenzo,20.0


#### Consulta 2
Lista as métricas de todos os jogadores que participaram de uma determinada partida, ordenando alfabeticamente.

In [5]:
id_pelada = 9

df02 = pd.read_sql_query("SELECT nome_pessoa, Pontos, Nota, G, GC " +
                         "FROM Jogador NATURAL JOIN Pessoa " +
                         "WHERE Jogador.id_pelada = " + str(id_pelada) + " " +
                         "ORDER BY nome_pessoa;", conn)

        
df02

,nome_pessoa,Pontos,Nota,G,GC
0,Emanuelly,26,None,1,0
1,Henrique,4,None,3,0
2,Joaquim,29,None,2,1
3,Laura,44,None,2,0
4,Lorena,21,None,1,0
5,Samuel,14,None,2,0
6,Sophia,30,None,3,0


#### Consulta 3
Lista todos os grupos dos quais determinado usuário é administrador.

In [6]:
#pessoa desejada
email = 'helena@gmail.com'

df06 = pd.read_sql_query("SELECT Grupo_de_pelada.nome, Grupo_de_pelada.descricao " +
                         "FROM Pessoa JOIN Grupo_de_Pelada ON Pessoa.id_pessoa = Grupo_de_Pelada.id_adm " +
                         "WHERE Pessoa.email = '" + email + "';", conn)

        
df06

,nome,descricao
0,Girls,


#### Consulta 4
Lista todas as peladas de um grupo que estão compreendidas em um determinado intervalo de tempo

In [7]:
id_grupo_de_pelada = 1 #grupo desejado
data_inicio = '2019-06-04 00:00:00' #formato AAAA-MM-DD HH:MM:SS
data_fim = '2019-12-30 00:00:00' #formato AAAA-MM-DD HH:MM:SS

df04 = pd.read_sql_query("SELECT lugar, preco, inicio, fim " +
                         "FROM Grupo_de_pelada NATURAL JOIN Pelada " +
                         "WHERE id_grupo_de_pelada = " + str(id_grupo_de_pelada) + " AND " +
                             "(inicio BETWEEN '"+ data_inicio +"' AND '" + data_fim + "' ) " +
                         "ORDER BY inicio;", conn)

        
df04

,lugar,preco,inicio,fim
0,Lugar5,8.0,2019-06-10 22:42:59,2019-06-11 00:42:30
1,Lugar28,3.0,2019-08-31 02:42:33,2019-08-31 04:42:04
2,Lugar29,9.0,2019-11-27 02:36:38,2019-11-27 04:36:09


#### Consulta 5
Lista todas vaquinhas de um grupo que estão compreendidas em um determinado intervalo de tempo

In [8]:
id_grupo_de_pelada = 1 #grupo desejado
data_inicio = '2019-06-04 00:00:00' #formato AAAA-MM-DD HH:MM:SS
data_fim = '2019-08-30 00:00:00' #formato AAAA-MM-DD HH:MM:SS

df05 = pd.read_sql_query("SELECT motivo, prazo " +
                         "FROM Grupo_de_pelada NATURAL JOIN Vaquinha " +
                         "WHERE id_grupo_de_pelada = " + str(id_grupo_de_pelada) + " AND " +
                             "(prazo BETWEEN '"+ data_inicio +"' AND '" + data_fim + "' ) " +
                         "ORDER BY prazo;", conn)

        
df05

,motivo,prazo
0,camisa,2019-08-11 00:00:00


#### Consulta 6
Lista todos os grupos de pelada a que uma pessoa pertence.

In [9]:
#pessoa desejada
email = 'isabella@gmail.com'

df06 = pd.read_sql_query("SELECT Grupo_de_pelada.nome, Grupo_de_pelada.descricao " +
                         "FROM Pessoa NATURAL JOIN Participa_grupo_pelada NATURAL JOIN Grupo_de_pelada " +
                         "WHERE Pessoa.email = '" + email + "';", conn)

        
df06

,nome,descricao
0,Girls,
1,Vodka,
2,Catuaba,


#### Consulta 7
Lista as últimas partidas que o jogador participou, o grupo de pelada em que a partida foi jogada, a data e a hora da partida, e as métricas do jogador na partida em questão, mostrando a mais recentes primeiro. 

Obs: Os dados em questão são fictícios. Por isso exitem resultados com datas futuras. No entanto, uma linha na tabela Jogador só será criada após a partida. Assim, apenas serão retornados valores de jogos que já ocorreram (e não que vão ocorrer, como aconteceu no exemplo abaixo).

In [10]:
#pessoa desejada
email = 'manuela@gmail.com'

df07 = pd.read_sql_query("SELECT Grupo_de_pelada.nome, Pelada.lugar, Pelada.preco, Pelada.inicio, Pontos, Nota, G, GC " +
                         "FROM Pessoa NATURAL JOIN Jogador NATURAL JOIN Pelada NATURAL JOIN Grupo_de_pelada " +
                         "WHERE Pessoa.email = '" + email + "'"+
                         "ORDER BY inicio desc;", conn)

        
df07

,nome,lugar,preco,inicio,Pontos,Nota,G,GC
0,Vodka,Lugar10,8.0,2020-04-16 08:12:47,13,None,0,0
1,Girls,Lugar5,8.0,2019-06-10 22:42:59,43,None,3,0
2,Mista,Lugar11,8.0,2019-04-27 20:13:45,26,None,2,0
3,Girls,Lugar12,4.0,2019-04-17 03:12:18,5,None,1,0
4,Vodka,Lugar25,14.0,2019-03-27 04:34:46,4,None,0,0
5,Vodka,Lugar4,5.0,2018-09-09 17:47:10,47,None,0,0


#### Consulta 8
Obtém todas as pessoas parcitipantes de uma vaquinha individual e a coluna de personalização (resposta) correspondente.

In [11]:
id_vaquinha_individual = 1

df08 = pd.read_sql_query("SELECT nome_pessoa, pergunta_personalizada, valor, resposta " +
                         "FROM Pessoa NATURAL JOIN Vaquinha_Individual_Pessoa NATURAL JOIN Vaquinha_Individual " +
                         "WHERE id_vaquinha_individual = " + str(id_vaquinha_individual) + " " +
                         "ORDER BY nome_pessoa;", conn)

df08

,nome_pessoa,pergunta_personalizada,valor,resposta
0,Arthur,Nome na camisa,25.0,Arthur
1,Benjamin,Nome na camisa,25.0,Benjamin
2,Davi,Nome na camisa,25.0,Davi
3,Enzo,Nome na camisa,25.0,Enzo
4,Gabriel,Nome na camisa,25.0,Gabriel
5,Guilherme,Nome na camisa,25.0,Guilherme
6,Heitor,Nome na camisa,25.0,Heitor
7,Heloisa,Nome na camisa,25.0,Heloisa
8,Isadora,Nome na camisa,25.0,Isadora
9,Joaquim,Nome na camisa,25.0,Joaquim


#### Consulta 9
Exibe a peladas com maior número total de gols feitos na partida, dentre as  quais determinado jogador participou.

In [4]:
email = 'beatriz@gmail.com'

df09 = pd.read_sql_query("SELECT Grupo_de_Pelada.nome, lugar, inicio, sum(G) as Gols_na_partida  " +
                         "FROM Jogador NATURAL JOIN Pelada NATURAL JOIN Grupo_de_Pelada " +
                         "WHERE id_pelada in (SELECT id_pelada FROM Jogador NATURAL JOIN Pessoa WHERE email = '" + email + "') " + 
                         "GROUP BY id_pelada " +
                         "ORDER BY Gols_na_partida desc limit(1);", conn)

df09

,nome,lugar,inicio,Gols_na_partida
0,Girls,Lugar28,2019-08-31 02:42:33,12


#### Consulta 10
Contabiliza o total já arrecadado em uma vaquinha coletiva.

In [13]:
df10 = pd.read_sql_query("SELECT motivo, prazo, sum(valor_pago) as total_pago, valor_total " +
                         "FROM (Vaquinha JOIN Vaquinha_Coletiva ON Vaquinha.id_vaquinha =  Vaquinha_Coletiva.id_vaquinha_coletiva)"+
                             " NATURAL JOIN Vaquinha_Coletiva_Pessoa " +
                          "GROUP BY id_vaquinha_coletiva;", conn)

df10

,motivo,prazo,total_pago,valor_total
0,churrasco,2019-06-10 00:00:00,200.0,240.0
1,aluguel campo,2019-07-03 00:00:00,80.0,100.0
2,churrasco,2019-10-02 00:00:00,240.0,300.0


## Consultas do tipo relatório

#### Relatório 1
Relatório que contém todos os grupos de pelada e suas médias de gols, gols contra, faltas, roubadas de bola, etc, apurado para todas as partidas.

In [14]:
rel1 = pd.read_sql_query("SELECT nome as Nome_Grupo, count(*) as Num_Peladas, avg(P) as Media_pontos,  " +
                                 "avg(G) as Media_gols, avg(GC) as media_gols_contra " +
                         "FROM Grupo_de_Pelada NATURAL JOIN "+
                               "(SELECT id_grupo_de_pelada, id_pelada, sum(Pontos) as P, sum (G) as G, sum(GC) as GC "+
                                "FROM Pelada NATURAL JOIN Jogador "+
                                "GROUP BY id_pelada )" +
                         "GROUP BY id_grupo_de_pelada;", conn)

rel1

,Nome_Grupo,Num_Peladas,Media_pontos,Media_gols,media_gols_contra
0,Girls,5,125.600,7.80,0.400000
1,Boys,2,101.000,8.50,0.000000
2,Mista,4,105.750,7.75,2.750000
3,Cerveja,6,126.000,7.50,0.333333
4,Vodka,8,88.875,5.75,0.250000
5,Catuaba,5,125.600,9.80,0.400000


#### Relatório 2
Relatório que contém todos os participantes de um grupo e suas médias de gols, gols contra, faltas, roubadas de bola, etc. Tal média é apurada através das partidas organizadas somente pelo grupo em questão.

In [15]:
id_grupo_de_pelada = 5

rel2 = pd.read_sql_query("SELECT nome_pessoa, avg(Pontos) as Media_Pontos, avg(G) as Media_gols, avg(GC) as Media_gols_contra " +
                         "FROM Grupo_de_Pelada NATURAL JOIN Pelada NATURAL JOIN Jogador NATURAL JOIN Pessoa "+
                         "WHERE  id_grupo_de_pelada = " + str(id_grupo_de_pelada) + " " + 
                         "GROUP BY id_pessoa " +
                         "ORDER BY nome_pessoa;", conn)

rel2

,nome_pessoa,Media_Pontos,Media_gols,Media_gols_contra
0,Beatriz,12.000000,1.000000,0.500000
1,Gabriel,33.500000,1.500000,0.000000
2,Giovanna,21.333333,1.000000,0.333333
3,Heitor,29.500000,1.000000,0.000000
4,Isabella,8.333333,1.000000,0.000000
5,Julia,27.666667,2.333333,0.000000
6,Lara,14.500000,2.500000,0.000000
7,Luiza,35.333333,1.333333,0.000000
8,Manuela,21.333333,0.000000,0.000000
9,Matheus,27.500000,1.000000,0.000000


#### Relatório 3
Relatório que mostra a evolução da pontuação do jogador mensalmente.

In [16]:
email = 'manuela@gmail.com'

rel3 = pd.read_sql_query("SELECT strftime('%m/%Y', inicio) as Mes, avg(Pontos) as Media_Pontos, avg(G) as Media_gols, avg(GC) as Media_gols_contra " +
                         "FROM Jogador NATURAL JOIN Pessoa NATURAL JOIN Pelada "+
                         "WHERE  email = '" + email + "' "+
                         "GROUP BY strftime('%m/%Y', inicio) " +
                         "ORDER BY strftime('%Y', inicio), strftime('%m', inicio);", conn)

rel3

,Mes,Media_Pontos,Media_gols,Media_gols_contra
0,09/2018,47.0,0.0,0.0
1,03/2019,4.0,0.0,0.0
2,04/2019,15.5,1.5,0.0
3,06/2019,43.0,3.0,0.0
4,04/2020,13.0,0.0,0.0
